# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* 		сalls — количество звонков,


* 		minutes — суммарная длительность звонков в минутах,


* 		messages — количество sms-сообщений,


* 		mb_used — израсходованный интернет-трафик в Мб,


* 		is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Привет, мы еще не знакомы, но если не против, давай на ты?=)

Не мог/могла бы ты порекомендовать книгу по машинному обучению для новичка. Книг много разных, но что из этого разнообразия стоит читать? Особенно хочется разобратся в параметрах моделей и способах повышениы точности предсказаний.

Была мысль добавить новые признаки (на основе группировки или среднего, например) в датасет для увелеченя точности предсказания, но не знаю корректно ли будет такое действие. В интернете пишут разное..

## Откройте и изучите файл

**Подгрузим библиотеки.**

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier


**Теперь откроем датасет и посмотрим на его содержимое (по условию задания предобработка данных не требуется).**

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

**В датасете нет пропусков, названия колонк соответствуют snake_case, типы данных тоже в порядке.**

## Разбейте данные на выборки

**В случае, когда тестовая выборка не спрятана, данные раздеяют в пропорции 3:1:1 (тренировочная выборка:валидационная: тестовая).**

**Для этого разделим первый раз общий датасет в пропорции 60% для тренировочной выборки, а 40% для валидационной и тестовой.**

**После разделим 40% на 20% для валидационной и 20% для тестовой выборки.**

In [5]:
df_train, common_for_valid_and_test = train_test_split(df, test_size=0.4, random_state=12345)

In [6]:
df_valid, df_test = train_test_split(common_for_valid_and_test, test_size=0.5, random_state=12345)

**Посмотрим на получившуюся размерность для каждой выборки.**

In [7]:
for name_of_df, df in {'TRAIN': df_train, 'VALID': df_valid, 'TEST': df_test}.items():
    print(f'\nРазмер выборки {name_of_df} -', df.shape)


Размер выборки TRAIN - (1928, 5)

Размер выборки VALID - (643, 5)

Размер выборки TEST - (643, 5)


**В общих признаках избавимся от целевого признака, его запишем в отдельную переменную для каждой выборки соотвественно.**

In [8]:
features_train = df_train.drop('is_ultra', axis=1)
train_target = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop('is_ultra', axis=1)
test_target = df_test['is_ultra']

### Вывод

**Общий датасет был разделен в пропорции 3:1:1 на:**

1. **Тренировчную выборку для обучения**


2. **Валидационную выборку для нахождения лучших параметров**


3. **Тестовую выборку для оценки итого качества работы модели.**

## Исследуйте модели

**Создадим словарь, в котором в дальнейшем будут храниться лучшие параметры для моделей.**

In [9]:
best_params_for_model = {}

### Рассмотрим модель Дерево Решений (DecisionTreeClassifier).

**Обучать модель будем на тренировочных данных, а на валидационной выборке будем искать лучшие гиперпараметры.**



In [10]:
%%time
best_accuracy_for_decision_tree = 0
for depth in range(1,11, 1):
    for min_split in range(2,11, 1):
        for min_leaf in range(1,11, 1):
            for crit in ['gini', 'entropy']:
                for split in ['best', 'random']:
                    model_decision_tree = DecisionTreeClassifier(random_state=12345, 
                                                                 criterion=crit,
                                                                 splitter=split,
                                                                 max_depth=depth,
                                                                 min_samples_split=min_split,
                                                                 min_samples_leaf=min_leaf)
                    model_decision_tree.fit(features_train, train_target)
                    accuracy = model_decision_tree.score(features_valid, target_valid)
#                     print('Accuracy -', accuracy, 'Params', [crit, split, depth, min_split, min_leaf])
                    if (accuracy > best_accuracy_for_decision_tree):
                        best_accuracy_for_decision_tree = accuracy
                        best_params_for_model['Decision_Tree'] = {'criterion': crit,
                                                                  'splitter': split,
                                                                  'max_depth': depth,
                                                                  'min_samples_split': min_split,
                                                                  'min_samples_leaf': min_leaf}

CPU times: user 13.5 s, sys: 50.7 ms, total: 13.5 s
Wall time: 13.5 s


### Рассмотрим модель Случайный Лес (RandomForestClassifier).

**Обучим модель и переберем ее параметры.**

**Но сначала локализуем диапазон значений количества оценщиков.**

**Полученное количество оценщиков, для которого достигается максимальная доля правильных ответов, изучим подробнее при переборе гиперпараметров модели ниже.**

In [11]:
%%time
best_accuracy_for_random_forest_in_range_1000 = 0
recommended_depth_value = 0
best_num_of_tress_in_range_1000 = 0
for n_trees in range(100, 1000, 100):
    for depth in range(5,11,1):
        model_random_forest = RandomForestClassifier(random_state=12345, 
                                                     n_estimators=n_trees,
                                                     max_depth=depth)
        model_random_forest.fit(features_train, train_target)
        accuracy = model_random_forest.score(features_valid, target_valid)
        if accuracy > best_accuracy_for_random_forest_in_range_1000:
            best_accuracy_for_random_forest_in_range_1000 = accuracy
            best_num_of_tress_in_range_1000 = n_trees
            recommended_depth_value = depth
print(f'\nЛучший показатель доли правильных ответов на диапазоне до 1000 - {best_accuracy_for_random_forest_in_range_1000}')            
print(f'\nЛучший показатель кол-ва оценщиков в диапазоне до 1000 - {best_num_of_tress_in_range_1000}') 
print(f'\nРекомендуемая глубина деревьев - {recommended_depth_value}')


Лучший показатель доли правильных ответов на диапазоне до 1000 - 0.8040435458786936

Лучший показатель кол-ва оценщиков в диапазоне до 1000 - 300

Рекомендуемая глубина деревьев - 7
CPU times: user 1min 8s, sys: 412 ms, total: 1min 8s
Wall time: 1min 8s


**Максимальная доля правильных ответов достигается при количестве оценщиков равным 300, рассмторим этот дипазон подробнее, уменьшим шаг до 10 и сузив диапазон вокруг 300 для поиска лучшего значения доли правльных ответов.**

In [12]:
%%time
best_accuracy_for_random_forest = 0
for n_trees in range((best_num_of_tress_in_range_1000 - 30),
                     (best_num_of_tress_in_range_1000 + 30), 10):
    for crit in ['gini', 'entropy']:
        for depth in range((recommended_depth_value - 1), 
                           (recommended_depth_value + 1), 1):
            for value_for_bootstrap in [True, False]:
                for value_for_start in [True, False]:
                    model_random_forest = RandomForestClassifier(random_state=12345, 
                                                                 n_estimators=n_trees,
                                                                 criterion=crit,
                                                                 max_depth=depth,
                                                                 bootstrap = value_for_bootstrap,
                                                                 warm_start=value_for_start)
                    model_random_forest.fit(features_train, train_target)
                    accuracy = model_random_forest.score(features_valid, target_valid)
#                     print('Accuracy -', accuracy, 'Params', [n_trees, crit, depth, value_for_bootstrap, value_for_start])
                    if (accuracy > best_accuracy_for_random_forest):
                        best_accuracy_for_random_forest = accuracy
                        best_params_for_model['Random_Forest'] = {'n_estimators': n_trees,
                                                                  'criterion': crit,
                                                                  'max_depth': depth,
                                                                  'bootstrap': value_for_bootstrap,
                                                                  'warm_start': value_for_start}

CPU times: user 1min 20s, sys: 416 ms, total: 1min 21s
Wall time: 1min 21s


### Рассмотрим модель Логистическая Регрессия (LogisticRegression)

**Обучим модель и переберем гиперпараметры.**

In [13]:
%%time
best_accuracy_for_logistic_regression = 0
for value_of_solver in ['newton-cg', 'liblinear']: 
    model_logistic_regression = LogisticRegression(random_state=12345, 
                                                   solver = value_of_solver)
    model_logistic_regression.fit(features_train, train_target)
    model_logistic_regression.score(features_valid, target_valid)
#     print('Accuracy -', accuracy, 'Params', [value_of_solver])
    if (accuracy > best_accuracy_for_logistic_regression):
        best_accuracy_for_logistic_regression = accuracy
    best_params_for_model['Logistic_Regression'] = {'solver': value_of_solver}

CPU times: user 102 ms, sys: 0 ns, total: 102 ms
Wall time: 104 ms


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

### Вывод

In [14]:
print(f'\nЛучший результат для Desicion Tree - {best_accuracy_for_decision_tree}')
print(f'Параметры модели Desicion Tree - {best_params_for_model["Decision_Tree"]}')
print(f'\nЛучший результат для Random Forest - {best_accuracy_for_random_forest}')
print(f'Параметры модели Random Forest - {best_params_for_model["Random_Forest"]}')
print(f'\nЛучший результат для Logistic Regression - {best_accuracy_for_logistic_regression}')
print(f'Параметры модели Logistic Regression - {best_params_for_model["Logistic_Regression"]}')


Лучший результат для Desicion Tree - 0.8040435458786936
Параметры модели Desicion Tree - {'criterion': 'gini', 'splitter': 'random', 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 3}

Лучший результат для Random Forest - 0.8055987558320373
Параметры модели Random Forest - {'n_estimators': 320, 'criterion': 'gini', 'max_depth': 7, 'bootstrap': True, 'warm_start': True}

Лучший результат для Logistic Regression - 0.7900466562986003
Параметры модели Logistic Regression - {'solver': 'liblinear'}


**Лучший результат показала модель RandomForest, а самой быстрой стала LogisticRegression.**

## Проверьте модель на тестовой выборке

**Теперь последовательно создадим и обучим все три модели с их лучшими параметрами на тренировочной выборке и оценим готовые модели на тестовой выборке.**

In [15]:
model_decision_tree_for_test = DecisionTreeClassifier(random_state=12345, 
                                                     criterion=best_params_for_model["Decision_Tree"]['criterion'],
                                                     splitter=best_params_for_model["Decision_Tree"]['splitter'],
                                                     max_depth=best_params_for_model["Decision_Tree"]['max_depth'],
                                                     min_samples_split=best_params_for_model["Decision_Tree"]['min_samples_split'],
                                                     min_samples_leaf=best_params_for_model["Decision_Tree"]['min_samples_leaf'])
model_decision_tree_for_test.fit(features_train, train_target)
test_accuracy_for_decision_tree = model_decision_tree_for_test.score(features_test, test_target)

In [16]:
model_random_forest_for_test = RandomForestClassifier(random_state=12345,
                                                     n_estimators=best_params_for_model["Random_Forest"]['n_estimators'],
                                                     criterion=best_params_for_model["Random_Forest"]['criterion'],
                                                     max_depth=best_params_for_model["Random_Forest"]['max_depth'],
                                                     bootstrap=best_params_for_model["Random_Forest"]['bootstrap'],
                                                     warm_start=best_params_for_model["Random_Forest"]['warm_start'])
model_random_forest_for_test.fit(features_train, train_target)
test_accuracy_for_random_forest = model_random_forest_for_test.score(features_test, test_target)

In [17]:
model_logistic_regression_for_test = LogisticRegression(random_state=12345,
                                                       solver=best_params_for_model["Logistic_Regression"]['solver'])
model_logistic_regression_for_test.fit(features_train, train_target)
test_accuracy_for_logistic_regression = model_logistic_regression_for_test.score(features_test, test_target)

### Вывод

In [18]:
print(f'\n Доля правильных ответов на тестовой выборке для Desicion Tree - {test_accuracy_for_decision_tree}')
print(f'\n Доля правильных ответов на тестовой выборке для Random Forest - {test_accuracy_for_random_forest}')
print(f'\n Доля правильных ответов на тестовой выборке для Logistic Regression - {test_accuracy_for_logistic_regression}')


 Доля правильных ответов на тестовой выборке для Desicion Tree - 0.8009331259720062

 Доля правильных ответов на тестовой выборке для Random Forest - 0.7978227060653188

 Доля правильных ответов на тестовой выборке для Logistic Regression - 0.7402799377916018


**Лучше всего себя проявила модель DecisionTree. RandomForest показали результат хуже, возможно модель переобучена.**

## (бонус) Проверьте модели на адекватность

**Для проверки адекватности моделей создадим простейшую (dummy) модель, которая всегда предсказывает наиболее часто встречающийся класс. Так мы получим контрольные данные для сравнительной оценки построенных моделей (более сложных). Сравнив значение доли правильных ответов dummy-модели с значением доли правильных ответов построенных моделей можно оценить адекватоность моделей.**

In [19]:
model_dummy = DummyClassifier(strategy="most_frequent", random_state=12345)
model_dummy.fit(features_train, train_target)
accuracy_for_model_dummy = model_dummy.score(features_test, test_target)

In [20]:
print(f'Доля правильных ответов для простейшей модели (Dummy Model) - {accuracy_for_model_dummy}')
print(f'\n Доля правильных ответов на тестовой выборке для Desicion Tree - {test_accuracy_for_decision_tree}')
print(f'\n Доля правильных ответов на тестовой выборке для Random Forest - {test_accuracy_for_random_forest}')
print(f'\n Доля правильных ответов на тестовой выборке для Logistic Regression - {test_accuracy_for_logistic_regression}')

Доля правильных ответов для простейшей модели (Dummy Model) - 0.6842923794712286

 Доля правильных ответов на тестовой выборке для Desicion Tree - 0.8009331259720062

 Доля правильных ответов на тестовой выборке для Random Forest - 0.7978227060653188

 Доля правильных ответов на тестовой выборке для Logistic Regression - 0.7402799377916018


### Вывод

**Все модели являются адекватными, так как их значение доли правильных ответов выше чем у простейшей модели.**

## Вывод

**Самой точной моделью c высокой долей правильных ответов является DecisionTreeClassifier, а самой быстрой LogisticRegression. Модель RandomForestClassifier показала самый высокие результаты на валидационной выборке, но на тестовой результаты оказались ниже чем у DecisionTreeClassifier, возможно модель была переобучена.**

**Все модели прошли проверку на адекватность(сравнение с результатами DummyСlassifier).**